# APLICACIONES EN CIENCIAS DE COMPUTACION¶

## Laboratorio 1: Corregir el Programa del Agente "Aspirador de 2 posiciones"

El presente notebook implementa un entorno de trabajo para el agente aspirador de dos posiciones. También encontras la implementacion del agente con un programa reflexivo simple. Las posiciones del entorno son denotadas como loc_A y loc_B, cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'. El agente debe limpiar su entorno, pero tiene un error en su programa. Tu tarea en este laboratorio sera identificar ese error y corregirlo. Al final del notebook deberas responder a las preguntas planteadas. 

# Clase <b>Thing</b>

  Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>.  

In [13]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

# Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [14]:
import collections
import random

class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

# Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location.

In [15]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepcion que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una accion 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localizacion especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningun agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localizacion location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

# Clase <b>VacuumEnvironment</b>

Esta clase implementa el entorno del aspirador de 2 posiciones: loc_A y loc_B. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'. Un agente en este entorno percibe su localizacion y el estado de la misma

In [16]:
loc_A, loc_B = (0, 0), (1, 0)  # The two locations for the Vacuum world

class VacuumEnvironment(Environment):

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}

    def thing_classes(self):
        return [ReflexVacuumAgent]

    def percept(self, agent):
        """Retorna la posicion del agente y el estado de la posicion (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Cambia la posicion del agente y/o el estado de la posicion; Aumenta en 10 el desempeño del agente por cada
         aspiracion (accion 'suck') en una posicion con estado 'Dirty; Disminuye en 1 el desempeño por cada movida"""
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Devuelve una posicion aleatoria."""
        return random.choice([loc_A, loc_B])

# Agente Aspirador de 2 posiciones con Programa Reactivo Simple

Este agente es el agente aspirador de dos posiciones que usa un programa reactivo simple: realiza una accion basado en la percepción (posicion, estado) actual

In [17]:
def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        if status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'
    return Agent(program)

# Probando el agente aspirador en su entorno

In [39]:
"""Crea un agente reflexivo para el entorno del aspirador de 2 posiciones y lo coloca en posicion loc_A"""
a = ReflexVacuumAgent()
a.location = loc_A

"""Crea el entorno del aspirador de 2 posiciones con ambas posiciones en estado 'Dirty'"""
e = VacuumEnvironment()
e.status = {loc_A: 'Dirty',  loc_B: 'Dirty'}

"""Añade el agente creado al entorno"""
e.add_thing(a) 

"""Ejecuta el entorno 5 pasos y obtiene el desempeño del agente"""
e.run(5)
print("Desempeño del agente = {} ".format(a.performance)) 

Desempeño del agente = 17 


# Preguntas:

1) <b>Cual es el error del programa del agente y cual es su solución?<b> (12 puntos)

El error está en que limpia cuando estaba en Clean, entonces nunca limpiaba, ya que inicialmente los dos locations estaban en Dirty. La solución está en cambiar el status == 'Dirty' para que limpie, es decir, return 'Suck'.


 def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        if status == 'Clean':           //En esta línea se encuentra el error
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'
    return Agent(program)
    
/////SOLUCIÓN//////
 def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        if status == 'Dirty':       //Corrección
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Left'
    return Agent(program)

2) <b>Cual es el valor de performance del agente (corregido) despues de 5 pasos de ejecución del entorno?<b> (4 puntos)

El valor de performance del agente corregido es 17.

3) <b>Cual es el minimo numero de pasos que se tiene que hacer en el ambiente para asegurar que este este limpio y cual seria el valor de performance del agente?<b> (4 puntos)

El mínimo número de pasos es 3 para asegurar que el ambiente esté limpio. Y el valor de performance del agente es 19. Este es porque en 3 pasos logra obtener el mayor desempeño posible, mientras que después el desempeño va disminuyendo. 